Use `selenium` to scrape UNTC

11-17-2017.

---

In [1]:
import time

In [2]:
from selenium.webdriver.common.keys import Keys

Set date

In [3]:
yyyy = 1945

In [4]:
from_text = '01/01/%s'%yyyy
to_text = '12/31/%s'%(yyyy+20)

In [5]:
from_text, to_text

('01/01/1945', '12/31/1965')

---

In [35]:
from selenium import webdriver
browser = webdriver.Chrome()
url = 'https://treaties.un.org/Pages/AdvanceSearch.aspx?tab=UNTS&clang=_en'
browser.get(url)

#### Criteria

##### Search Treaty

https://stackoverflow.com/questions/7867537/selenium-python-drop-down-menu-option-value

In [36]:
from selenium.webdriver.support.ui import Select
select = Select(browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$drpSearchObj'))
select.select_by_visible_text('Treaty')

###### Select "Show Only Original Agreement"

In [57]:
browser.find_element_by_id('ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolderInnerPage_rdbtreaty_2').click()

##### Select filter by: Treaty type

In [58]:
select = Select(browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$drpAttribute'))

In [59]:
select.select_by_index(20)

###### Select "Open Multilateral"

In [60]:
browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$chkboxValues$2').click()

###### Match all these values

In [61]:
browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$Adv_srch2').click()

###### Add to criteria button

In [62]:
browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$btnAdd').send_keys(Keys.SPACE)

##### Set date

In [63]:
select = Select(browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$drpAttribute'))

In [64]:
select.select_by_visible_text('Date of Registration')

###### From and end date

https://stackoverflow.com/questions/39520708/using-selenium-on-calendar-date-picker

In [65]:
from selenium.webdriver.common.keys import Keys

In [66]:
if '3' in str(from_text):
    from_text = from_text.replace('3', Keys.NUMPAD3)
if '3' in str(to_text):
    to_text = to_text.replace('3', Keys.NUMPAD3)

In [69]:
print(from_text, to_text)

01/01/1945 12/1/1965


In [89]:
from_field = browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$txtFrom')
from_field.send_keys(Keys.SPACE)
from_field.clear()
from_field.send_keys(from_text)

In [90]:
from_field.get_attribute('value')

'01/01/1945'

In [91]:
to_field = browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$txtTo')
to_field.send_keys(Keys.SPACE)
to_field.clear()
to_field.send_keys(to_text)
#to_field.send_keys(Keys.ENTER)

In [92]:
to_field.get_attribute('value')

'12/31/1965'

In [93]:
browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$btnAdd').send_keys(Keys.SPACE)

##### Search button

In [33]:
browser.find_element_by_name('ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$btnSubmit').send_keys(Keys.SPACE)

---

#### Iterate

In [26]:
import pandas as pd

In [27]:
from bs4 import BeautifulSoup as Soup

In [28]:
column_names = ['href', 'title', 'reg_num', 'reg_date', 'type', 'con_date', 'vol']

In [29]:
base_url = 'https://treaties.un.org'

In [30]:
def parse_page(tr_list):
    #tr_list = [Soup(tr.text, 'lxml') for tr in tr_list]
    tmp = list()
    for tr in tr_list:
        tmp.append(list())
        for i, td in enumerate(tr.find_all('td')):
            if i == 0:
                tmp[-1].append(base_url+td.a['href'])
            tmp[-1].append(td.text.strip())
    return pd.DataFrame(tmp, columns=column_names)

In [31]:
df = pd.DataFrame(columns=column_names)

In [43]:
btn = browser.find_elements_by_tag_name('tbody')[3].find_element_by_tag_name('tr').find_elements_by_tag_name('td')[-1].find_element_by_tag_name('input')

In [44]:
print(btn.get_attribute('name'))
btn.get_attribute('disabled')

ctl00$ctl00$ContentPlaceHolder1$ContentPlaceHolderInnerPage$dgTreaty$ctl01$ctl14


In [47]:
while True:
    source = Soup(browser.page_source, 'lxml')
    tr_list = source.find_all('tbody')[3].find_all('tr')[3:-2]
    df = df.append(parse_page(tr_list), ignore_index=True)
    if btn.get_attribute('disabled') is not None:
        break
    time.sleep(pd.np.random.randint(5, 10))
    tbody = browser.find_elements_by_tag_name('tbody')[3]
    btn = tbody.find_element_by_tag_name('tr').find_elements_by_tag_name('td')[-1].find_element_by_tag_name('input')
    btn.click()

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

In [266]:
df.to_csv('./')

,href,title,reg_num,reg_date,type,con_date,vol
0,/Pages/showDetails.aspx?objid=0800000280058127...,United Nations Convention on the Use of Electr...,I-50525,01/03/2013,Open Multilateral,23/11/2005,2898
1,/Pages/showDetails.aspx?objid=0800000280343dac...,"Treaty on long-term good-neighbourliness, frie...",I-50518,01/02/2013,Open Multilateral,16/08/2007,2896
2,/Pages/showDetails.aspx?objid=080000028032d20f...,Treaty on cooperation among the member States ...,I-50095,03/09/2012,Open Multilateral,04/06/1999,2867
3,/Pages/showDetails.aspx?objid=080000028032d246...,Treaty among member States of the Commonwealth...,I-50100,03/09/2012,Open Multilateral,05/10/2007,2867
4,/Pages/showDetails.aspx?objid=08000002802e8dab...,The Istanbul Agreement on the Parliamentary As...,I-49912,02/07/2012,Open Multilateral,21/11/2008,2853
5,/Pages/showDetails.aspx?objid=08000002803296a6...,Southern Indian Ocean Fisheries Agreement,I-49647,21/06/2012,Open Multilateral,07/07/2006,2835
6,/Pages/showDetails.aspx?objid=08000002802da30a...,Nakhchivan Agreement on the establishment of t...,I-49924,02/07/2012,Open Multilateral,03/10/2009,2854
7,/Pages/showDetails.aspx?objid=080000028006822e...,"Maritime Labour Convention, 2006",I-51299,20/08/2013,Open Multilateral,23/02/2006,2952
8,/Pages/showDetails.aspx?objid=080000028029a8a3...,"International Cocoa Agreement, 2010",I-50115,01/10/2012,Open Multilateral,25/06/2010,2871
9,/Pages/showDetails.aspx?objid=080000028032bce0...,Free Trade Agreement between the United Mexica...,I-49661,12/06/2012,Open Multilateral,15/11/2003,2837


---

In [34]:
browser.close()